In [ ]:
import numpy as np
import pystan

In [ ]:
numPeople = 2
numModesPerPerson = 3
numSpectralCoeff = 5
numSamplesPerMode = 1000

In [ ]:
modeVariance = 10

modeMatrixList = [np.random.multivariate_normal(np.zeros(numSpectralCoeff), 
                                                modeVariance * np.identity(numSpectralCoeff), 
                                                size=numModesPerPerson) for i in range(numPeople)]
modeMatrixList

In [ ]:
dataList = [[np.random.multivariate_normal(modeMatrixList[j][i,:], 
                                          np.identity(numSpectralCoeff), 
                                          size=numSamplesPerMode) for i in range(numModesPerPerson)] 
            for j in range(numPeople)]

len(dataList), len(dataList[0]) # (numPeople, numModesPerPerson)

In [ ]:
data = np.random.permutation(np.concatenate([dataList[j][i] for i in range(numModesPerPerson) 
                                             for j in range(numPeople)]))
data.shape

In [ ]:
mixModel = '''
data {
  int<lower=1> K;          // number of mixture components
  int<lower=1> N;          // number of data points
  vector[5] y[N];               // observations
}
parameters {
  simplex[K] theta;          // mixing proportions
  ordered[5] mu;             // locations of mixture components
  cov_matrix[5] Sigma;
}
model {
  vector[K] log_theta = log(theta);  // cache log calculation
  mu ~ normal(0, 10);
  for (n in 1:N) {
      y[n] ~ multi_normal(mu, Sigma);
  }
}
'''

In [ ]:
mixModel_dat = {'K': 6,
                'N': data.shape[0],
                'y': data}

In [ ]:
sm = pystan.StanModel(model_code=mixModel)

In [ ]:
fit = sm.sampling(data=mixModel_dat, iter=1000, chains=4)